In [1]:
import pandas as pd
import os
import datetime


In [3]:
base_path = os.path.join('tcat-ny-us')
trips_df = pd.read_csv(os.path.join(base_path, 'trips.txt'))
stop_times_df = pd.read_csv(os.path.join(base_path, 'stop_times.txt'))
stops_df = pd.read_csv(os.path.join(base_path, 'stops.txt'))


In [4]:
route_id = 30; 
relevant_trips = trips_df[trips_df['route_id'] == route_id]
relevant_stop_times = stop_times_df[stop_times_df['trip_id'].isin(relevant_trips['trip_id'])]

current_time = datetime.datetime.now()
current_time_str = current_time.strftime('%H:%M:%S')
print(current_time_str)
# Get active trips for the current time
active_trips = (relevant_stop_times[
    (relevant_stop_times['departure_time'] <= current_time_str) & 
    (relevant_stop_times['arrival_time'] >= current_time_str)
])

# Get the first active trip_id
active_trip_id = active_trips['trip_id']
print(active_trip_id)


17:56:27
5468    t3AE-b1F48-slC
Name: trip_id, dtype: object


In [5]:
relevant_stop_times = stop_times_df[stop_times_df['trip_id'].isin(relevant_trips['trip_id'])]

ordered_stops = (relevant_stop_times[relevant_stop_times['trip_id'] == active_trip_id]
                .merge(stops_df, on='stop_id')
                .sort_values('stop_sequence')
                [['stop_name']]
                .drop_duplicates())

ordered_stops.shape

ValueError: Can only compare identically-labeled Series objects

In [23]:
output_file = os.path.join('tcat-ny-us', f'route_v2_{route_id}.txt')
ordered_stops['stop_name'].to_csv(output_file, index=False, header=False, quoting=None)

In [ ]:
relevant_stop_times = stop_times_df[stop_times_df['trip_id'].isin(relevant_trips['trip_id'])]

stops_df.head()

# ordered_stops = (relevant_stop_times
#                     .merge(stops_df, on='stop_id')
#                     .sort_values('stop_sequence')
#                     [['stop_name']]
#                     .drop_duplicates())
# output_file = os.path.join('tcat-ny-us', f'route_{route_id}.txt')
# ordered_stops['stop_name'].to_csv(output_file, index=False, header=False, quoting=None)



In [ ]:
def get_ordered_stops_for_route(route_id):
    # Filter trips for the specific route
    relevant_trips = trips_df[trips_df['route_id'] == route_id]
    
    # Get stop times for those trips
    relevant_stop_times = stop_times_df[stop_times_df['trip_id'].isin(relevant_trips['trip_id'])]
    
    # Join with stops and order by sequence
    ordered_stops = (relevant_stop_times
                    .merge(stops_df, on='stop_id')
                    .sort_values('stop_sequence')
                    [['stop_id', 'stop_name', 'stop_sequence']]
                    .drop_duplicates())
    
    return ordered_stops.to_dict('records')

In [9]:
def get_stops_by_trip(trip_id):
    try:
        # Get stop times for the specific trip
        trip_stop_times = stop_times_df[stop_times_df['trip_id'].astype(str) == str(trip_id)]
        
        # Get ordered stops by joining with stops table
        ordered_stops = (trip_stop_times
                        .merge(stops_df, on='stop_id')
                        .sort_values('stop_sequence')
                        [['stop_name']]
                        .drop_duplicates())
        
        if ordered_stops.empty:
            print(f"No stops found for trip_id: {trip_id}")
            return []
            
        d = ordered_stops.to_dict('records')
        for e in d:
            print(e['stop_name'].strip())
        
    except Exception as e:
        print(f"Error getting stops for trip_id {trip_id}: {str(e)}")
        return []


get_stops_by_trip('t730-b1B67-slD')

Ithaca Commons - Seneca St
Albany at Salvation Army
Ithaca Commons - Green St
State/MLK @ Stewart
State/MLK @ Quarry
College @ Mitchell
Collegetown Crossing
College @ Oak
Carpenter Hall
Baker Flagpole  - Slope Side
Milstein Hall
Thurston at Balch Hall
Jessup @ Triphammer
Robert Purcell Community Center
Jessup @ Pleasant Grove
Pleasant Grove @ Hanshaw
Triphammer @ Texas
Triphammer @ Winthrop
Triphammer at Kendal
Triphammer Marketplace
Cayuga Mall
Graham@Butler
The Shops at Ithaca Mall
